### Probate records data noodling
Data set from Gloria Main, emerita U Colorado:
http://eh.net/database/new-england-1631-1776-sample-of-18509-probates/

Code by Shane Landrum, <srl@cliotropic.org>


In [2]:
import pandas as pd
import numpy as np

data_file = 'Main-Main-New-Eng-probates-2013.xlsx'
                       
def probate_records():
    pr = pd.read_excel( data_file,
                        sheetname='main data set', 
                        header=0,
                        skiprows=4, 
                        skip_footer=19,
                        index_col=0, 
                        parse_cols='A:C,M:T')  # basic year and life cycle information
    return pr.fillna('')

def colonies():
    c = pd.read_excel(data_file,
                      sheetname='Some code values',
                      header=0,
                      index_col=0,
                      skiprows=4,
                      skip_footer=270,
                      parse_cols="A,B,C")
    return c.fillna('')


def counties():
    c = pd.read_excel(data_file,
                      sheetname='Some code values',
                      header=0,
                      index_col=0,
                      skiprows=15,
                      skip_footer=251,
                      parse_cols="A,B,C")
    return c.fillna('')

def towns():
    t = pd.read_excel(data_file,
                      sheetname='Some code values',
                      header=0,
                      index_col=0, 
                      skiprows=4,
                      skip_footer=2,
                      parse_cols="M:Q")
    return t.fillna('')


def bigtable():
    # This will take some time because the data file is 6.5MB.
    # merge tables to bring in county names.
    bigtable = probate_records().merge(counties(), how="left", left_on="county", right_index=True)
    # Move textual names for counties into the 'county' column, drop the badly-labeled 'This is' column
    bigtable['county'] = bigtable['This is']
    bigtable = bigtable.drop(['This is'], 1)

    # merge tables to bring in town names. 
    townlabels = towns()["This town is"].to_frame()
    bigtable = bigtable.merge(townlabels, how="left", left_on="town", right_index=True)
    # Move textual names for towns into the 'town' column, drop the badly-labeled 'This town is' column
    bigtable['town'] = bigtable['This town is']
    bigtable = bigtable.drop(['This town is'], 1)
    bigtable = bigtable.drop(['sample n ='], 1) # Not sure why this is getting added, but make it go away

    # merge tables to bring in colony names.
    bigtable = bigtable.merge(colonies(), how="left", left_on="colony", right_index=True)
    # Move textual names for colonies into the 'colony' column, drop the badly-labeled 'This is' column
    bigtable['colony'] = bigtable['This is']
    bigtable = bigtable.drop(['This is'], 1)
    bigtable = bigtable.drop(['Sample n ='], 1) # Not sure why this is getting added, but make it go away

    
    return(bigtable.fillna(""))

#print(colonies())
#counties()
#towns()
bigtable()

,year,age,county,town,born,xocc,sex,newlc,xtown,colony
row,,,,,,,,,,
1,1646,,Plymouth,Scituate,,4,0,3,1,Plymouth
2,1647,,Plymouth,,,6,0,5,,Massachusetts (excl. Boston)
3,1647,69,Plymouth,Plymouth,1578,5,0,5,2,Plymouth
4,1647,,Plymouth,,,5,0,3,,Massachusetts (excl. Boston)
5,1648,,Plymouth,,,6,0,3,,Massachusetts (excl. Boston)
6,1649,,Plymouth,,,5,0,2,,Massachusetts (excl. Boston)
7,1649,,Plymouth,,,5,0,1,,Massachusetts (excl. Boston)
8,1650,,Plymouth,,,5,0,,,Massachusetts (excl. Boston)
9,1650,,Plymouth,,,4,0,4,,Massachusetts (excl. Boston)
